# PUC Minas
## Prediçāo de Insuficiência Cardíaca
### Projeto Integrado – Inteligência Artificial e Aprendizado de Máquina

- **Autor**: Vinicio Silva Lima

Este notebook tem como objetivo apresentar todo o processo de desenvolvimento do modelo preditivo para a base de "Heart Failure Prediction Dataset" da plataforma [Kaggle](https://www.kaggle.com). A base pode ser encontrada por meio do seguinte [link](https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction).

### Import das bibliotecas

In [1]:
import pandas as pd
import numpy as np
from category_encoders.one_hot import OneHotEncoder

### Lendo a base de dados

In [2]:
#Lendo csv
df = pd.read_csv('heart.csv')
#Verificando a quantidade de linhas, colunas, nulos e o tipo de cada varíavel
df.info()
##Verificando as 3 primeiras linhas do dataframe
df.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0


<p style="font-size:11pt;"> Podemos observar que nāo temos nenhuma variável com dados nulos. Apesar disso, vamos precisar categorizar essas variáveis em formato object para utilizar no modelo posteriormente.</p>

### Separando as variáveis númericas

In [32]:
#Selecionando as variáveis de formato numérico
atributos_numericos = df.select_dtypes(include=['int64', 'float64'])
#Verificando se as variáveis sāo numéricas
atributos_numericos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Age           918 non-null    int64  
 1   RestingBP     918 non-null    int64  
 2   Cholesterol   918 non-null    int64  
 3   FastingBS     918 non-null    int64  
 4   MaxHR         918 non-null    int64  
 5   Oldpeak       918 non-null    float64
 6   HeartDisease  918 non-null    int64  
dtypes: float64(1), int64(6)
memory usage: 50.3 KB


### Separando as variáveis em formato string e convertendo-as para formato categórico

In [33]:
#Selecionando as variáveis de formato object e transformando em categóricas
atributos_categoricos = df.select_dtypes(include=['object']).astype('category')
#Verificando se as variáveis sāo categóricas
atributos_categoricos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   Sex             918 non-null    category
 1   ChestPainType   918 non-null    category
 2   RestingECG      918 non-null    category
 3   ExerciseAngina  918 non-null    category
 4   ST_Slope        918 non-null    category
dtypes: category(5)
memory usage: 5.3 KB


### Aplicando métricas estatísticas para entender as variáveis numéricas

In [34]:
# métricas de tendência central
# transpor para facilitar a visualização
ct1 = pd.DataFrame(atributos_numericos.apply(np.mean)).T # média
ct2 = pd.DataFrame(atributos_numericos.apply(np.median)).T # mediana
# métricas de dispersão
d1 = pd.DataFrame(atributos_numericos.apply(np.std)).T # desvio-padrão
d2 = pd.DataFrame(atributos_numericos.apply(min)).T # mínimo
d3 = pd.DataFrame(atributos_numericos.apply(max)).T # máximo
d4 = pd.DataFrame(atributos_numericos.apply(lambda x: x.max() - x.min())).T # range
d5 = pd.DataFrame(atributos_numericos.apply(lambda x: x.skew())).T # assimetria
d6 = pd.DataFrame(atributos_numericos.apply(lambda x: x.kurtosis())).T # curtose
# concatenar as métricas
m = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index() # métricas
# nomear as colunas dos atributos concatenados
m.columns = ['Atributos' , 'Min', 'Max', 'Range', 'Média', 'Mediana', 'Desvio-Padrão', 'Assimetria',
           'Curtose']
#Mostra tabela com os resultados
m

,Atributos,Min,Max,Range,Média,Mediana,Desvio-Padrão,Assimetria,Curtose
0,Age,28.0,77.0,49.0,53.510893,54.0,9.427478,-0.195933,-0.386140
1,RestingBP,0.0,200.0,200.0,132.396514,130.0,18.504067,0.179839,3.271251
2,Cholesterol,0.0,603.0,603.0,198.799564,223.0,109.324551,-0.610086,0.118208
3,FastingBS,0.0,1.0,1.0,0.233115,0.0,0.422815,1.264484,-0.401960
4,MaxHR,60.0,202.0,142.0,136.809368,138.0,25.446463,-0.144359,-0.448248
5,Oldpeak,-2.6,6.2,8.8,0.887364,0.6,1.065989,1.022872,1.203064
6,HeartDisease,0.0,1.0,1.0,0.553377,1.0,0.497143,-0.215086,-1.958008


<p style="font-size:11pt;"> Podemos perceber que a variável de Cholesterol possui valores zerados e um desvio-padrāo considerável. A variável de Oldpeak está apresentando valores negativos, entāo será necessário entender se esses valores sāo válidos.</p>

### Filtrando as colunas numéricas e entendendo um pouco a distribuíçāo por quartil das variáveis

In [35]:
#Entendendo as variáveis numéricas
df.describe(include=['float64', 'int64'], percentiles=[0.01, .1, .25, .50, .75, .9, .99]).T

,count,mean,std,min,1%,10%,25%,50%,75%,90%,99%,max
Age,918.0,53.510893,9.432617,28.0,32.00,40.0,47.00,54.0,60.0,65.0,74.00,77.0
RestingBP,918.0,132.396514,18.514154,0.0,95.00,110.0,120.00,130.0,140.0,160.0,180.00,200.0
Cholesterol,918.0,198.799564,109.384145,0.0,0.00,0.0,173.25,223.0,267.0,305.0,411.49,603.0
FastingBS,918.0,0.233115,0.423046,0.0,0.00,0.0,0.00,0.0,0.0,1.0,1.00,1.0
MaxHR,918.0,136.809368,25.460334,60.0,77.17,103.0,120.00,138.0,156.0,170.0,186.00,202.0
Oldpeak,918.0,0.887364,1.066570,-2.6,-0.50,0.0,0.00,0.6,1.5,2.3,4.00,6.2
HeartDisease,918.0,0.553377,0.497414,0.0,0.00,0.0,0.00,1.0,1.0,1.0,1.00,1.0


<p style="font-size:11pt;"> Podemos observar que a variável de Cholesterol provavelmente possui outliers, já que o valor máximo apresentado se difere muito em relaçāo a média.</p>

### Concatenando os dados categoricos e numéricos

In [36]:
#Juntando as bases das variáveis númericas e categóricas
df = pd.concat([atributos_categoricos, atributos_numericos], axis=1)
#Verificando as 3 primeiras linhas do dataframe
df.head(3)

,Sex,ChestPainType,RestingECG,ExerciseAngina,ST_Slope,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
0,M,ATA,Normal,N,Up,40,140,289,0,172,0.0,0
1,F,NAP,Normal,N,Flat,49,160,180,0,156,1.0,1
2,M,ATA,ST,N,Up,37,130,283,0,98,0.0,0


### Filtrando colunas categoricas e aplicando o OneHotEncoder para classifica-las numericamente

In [37]:
#Filtrando as colunas categoricas
cols_encoding = df.select_dtypes(include='category').columns
#Passando para o OneHotEncoder as colunas categoricas
ohe = OneHotEncoder(cols=cols_encoding)
#Quebrando as variáveis categóricas em colunas númericas
encoded = ohe.fit_transform(df)
#Verificando as 3 primeiras linhas do dataframe
encoded.head(3)

,Sex_1,Sex_2,ChestPainType_1,ChestPainType_2,ChestPainType_3,ChestPainType_4,RestingECG_1,RestingECG_2,RestingECG_3,ExerciseAngina_1,...,ST_Slope_1,ST_Slope_2,ST_Slope_3,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
0,1,0,1,0,0,0,1,0,0,1,...,1,0,0,40,140,289,0,172,0.0,0
1,0,1,0,1,0,0,1,0,0,1,...,0,1,0,49,160,180,0,156,1.0,1
2,1,0,1,0,0,0,0,1,0,1,...,1,0,0,37,130,283,0,98,0.0,0
